# Abastecimiento de demanda

Dadas dos funciones de ¿precio?, asociadas a dos proveedores distintos, necesitamos cubrir una demanda específica

* $\Phi_{1}(x)$ es el precio de comprarle al proveedor 1, una cantidad $x$.
* $\Phi_{2}(x)$ es el precio de comprarle al proveedor 2, una cantidad $x$.

Debemos minimizar :

$$\min_{a_{ij}} \sum_{i,j} a_{ij} |x_i-y_j|$$

sujeto a:

$$\sum_j a_{ij} = p_i \quad \forall i=1,\ldots,m,$$

$$\sum_i a_{ij} = q_j \quad \forall j=1,\ldots,n.$$

In [3]:
## Para instalar los paquetes del Project.toml la primera vez
#using Pkg; Pkg.instantiate()
#using Pkg; Pkg.precompile()

Precompiling project...
  ✓ SuiteSparse_jll
  ✓ Sundials_jll
  ✓ Sundials
  ✓ DifferentialEquations
  4 dependencies successfully precompiled in 13 seconds (255 already precompiled, 2 skipped during auto due to previous errors)


In [1]:
import Pkg; Pkg.add("GLPK")

In [2]:
using JuMP, GLPK, Ipopt

#m=3
x=[1.0;2.0;3.0]
p=[0.2;0.4;0.4]

#n=2
y=[0;4.0]
q=[0.5;0.5]

#model = JuMP.Model(GLPK.Optimizer)
model = JuMP.Model(Ipopt.Optimizer)


m=length(x)
n=length(y)

#Calculo los pesos |xi-yj|
W=abs.(x*ones(1,n) - ones(m,1)*y')

@variable(model,A[1:m,1:n]>=0)

conp = @constraint(model, sum(A, dims=2).==p)
conq = @constraint(model, sum(A, dims=1).==q')

@objective(model,Min, sum( A.*W ))

model

LoadError: ArgumentError: Package JuMP not found in current path:
- Run `import Pkg; Pkg.add("JuMP")` to install the JuMP package.


In [13]:
optimize!(model)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.13.4, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       12
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        6
                     variables with only lower bounds:        6
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equal

In [14]:
objective_value(model)

1.3999999650258146

In [15]:
#Extraigo el valor optimo
value.(A)

3×2 Matrix{Float64}:
 0.2  0.0
 0.3  0.1
 0.0  0.4

In [16]:
#Extraigo los multiplicadores de p
lambda = dual.(conp)

3×1 Matrix{Float64}:
 -10.984989336764812
  -9.984989332634358
 -10.984989313845889

In [17]:
#Extraigo los multiplicadores de q
mu = dual.(conq)

1×2 Matrix{Float64}:
 11.985  11.985

In [18]:
W

3×2 Matrix{Float64}:
 1.0  3.0
 2.0  2.0
 3.0  1.0

In [19]:
lambda * ones(1,n) + ones(m,1)*mu

3×2 Matrix{Float64}:
 1.0  1.0
 2.0  2.0
 1.0  1.0